## Imports

In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

import utils.config as config

plt.rcParams["figure.dpi"] = 180

# define prediction horizon and type of dimensionality reduction
PRED_HORIZON = 64
DIM_REDUCTION = "SVD"       # one of ("SVD" / "VAE")

# define paths
FC_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study", f"pred_horizon_{PRED_HORIZON}")
OUTPUT_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study")

## Plot Study Results

In [ ]:
# load study results
study_results = pt.load(join(FC_PATH, "study_results.pt"))
param_combinations = list(study_results.keys())

# find parameter combinations of study and extract test loss
hidden_size = np.unique([int(param_set.split('_')[1]) for param_set in param_combinations])
n_hidden = np.unique([int(param_set.split('_')[2]) for param_set in param_combinations])

X, Y = np.meshgrid(hidden_size, n_hidden)

In [ ]:
# Calculate the mean and standard deviation of test loss for each epoch for each parameter combination
train_means = {}
test_means = {}

for param_combination in study_results:
    all_test_losses = []
    all_train_losses = []
    for df in study_results[param_combination]:
        all_test_losses.append(df["val_loss"])
        all_train_losses.append(df["train_loss"])
    
    # Pad shorter training sequences with NaNs to ensure equal lengths
    all_test_losses_padded = np.array([np.pad(loss, (0, 500 - len(loss)), mode='constant', constant_values=np.nan) for loss in all_test_losses])
    all_train_losses_padded = np.array([np.pad(loss,(0, 500 - len(loss)), mode='constant', constant_values=np.nan) for loss in all_train_losses])
    
    # Calculate mean and std while ignoring NaN values
    test_means[param_combination] = np.nanmean(all_test_losses_padded, axis=0)
    train_means[param_combination] = np.nanmean(all_train_losses_padded, axis=0)

# Create the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3), sharey = True, sharex = True)

# Plot mean test loss with 2 sigma confidence interval for each latent size
for size in study_results:
    ax1.plot(np.arange(1, config.VAE_epochs + 1), train_means[size], label=f'Latent Size {size}')
    ax2.plot(np.arange(1, config.VAE_epochs + 1), test_means[size], label=f'Latent Size {size}')

ax1.set_xlabel("epoch")
ax1.set_yscale("log")
ax1.set_title("Train MSE Mean")
ax2.set_xlabel("epoch")
ax2.set_yscale("log")
ax2.set_title("Test MSE Mean")
ax1.legend(loc=1, bbox_to_anchor=(1,1))
ax1.grid()
ax2.grid()
plt.xlim(0, config.VAE_epochs)
plt.tight_layout()
plt.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_loss_mean.png"), bbox_inches="tight")

In [ ]:
final_test_mean = np.array([test_means[param_comb][-10:].mean() for param_comb in test_means.keys()])

# Create a heatmap
plt.figure(figsize=config.standard_figsize_2)
heatmap = plt.pcolormesh(X, Y, final_test_mean.reshape(X.shape), cmap='viridis', norm=LogNorm())

plt.colorbar(heatmap, label='Test Loss')
plt.xlabel('Number of Hidden Layers')
plt.ylabel('Hidden Layer Neurons')
plt.title('Parameter Study: Test Loss Heatmap')
plt.xticks(hidden_size)
plt.yticks(n_hidden)
plt.gca().invert_yaxis()  # Invert y-axis to have larger values at the top
plt.tight_layout()
plt.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_param_study.png"), bbox_inches="tight")